# Running a Molecular Dynamics Simulation with Hoomd-Polymers

## Overview:
In this tutorial, we will run a molecular dynamics simulation of Polyphenylene sulfide molecules using the Hoomd-Polymers package. We will use the [`HOOMD-blue`](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulation engine to run the simulation and the `hoomd_organics` package to initialize the system of polymer configuration.

In summary, the `hoomd-polymers` package has three main classes:

-  `Molecule`: This class is used to define the structure of a molecule (for example the structure of a polymer built from a monomer).

- `System`: This class is used to define the system of molecules (for example a system of polymers) in a box and creates the initial `gsd` snapshot of the system. It also applies the forcefiled to the system and prepares the required forces for the simulation.

- `Simulation`: This class is used to run the simulation using the `HOOMD-blue` simulation engine. In order to initialize a simulation, a `gsd` snapshot of the system and a list of `Hoomd` forces are required.

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from hoomd_organics.library import PolyEthylene, OPLS_AA
from hoomd_organics import Pack
from hoomd_organics.modules.welding import SlabSimulation, Interface

import gsd
import unyt as u

### Step 1: Creating a "Slab"



In [5]:
molecule = PolyEthylene(num_mols=40, lengths=15)
system = Pack(molecules=molecule, force_field=OPLS_AA(), density=1.1, r_cut=2.5, auto_scale=True, remove_charges=True, remove_hydrogens=True)


KeyboardInterrupt



In [ ]:
help(SlabSimulation)

In [ ]:
sim = SlabSimulation.from_system(
    system=system,
    interface_axis=(1,0,0),
    gsd_file_name="slab_x_interface.gsd",
)

In [ ]:
sim.forces

In [ ]:
for wall in sim.forces[-1].walls:
    print(wall)

In [ ]:
sim.box_lengths_reduced

In [ ]:
sim.run_update_volume(final_density=1.2*u.g/(u.cm**3), n_steps=5e4, kT=5.0, period=100, tau_kt=0.001)

In [ ]:
sim.density.to(u.g/(u.cm**3))

In [ ]:
sim.run_NVT(kT=5.0, n_steps=5e4, tau_kt=0.001)

In [ ]:
sim.pickle_forcefield("polyethylene_ua_no_charges.pickle")

In [ ]:
# Visualize slab, wrapped and unwrapped

### Step 2: Creating an interface from the slab



In [ ]:
interface = Interface(gsd_file="slab_x_interface.gsd", interface_axis=(1, 0, 0), gap=0.1)

In [ ]:
print(type(interface.hoomd_snapshot))
print("Slab number of particles:", system.n_particles)
print("Interface number of particles:", interface.hoomd_snapshot.particles.N)
print()
print("Slab box lengths:", sim.box_lengths_reduced)
print("Interface box lengths:", interface.hoomd_snapshot.configuration.box[:3])

In [ ]:
with gsd.hoomd.open("interface.gsd", "w") as traj:
    traj.append(interface.hoomd_snapshot)

In [ ]:
weld_sim = 

### Step 2: Defining the System
In this step, we will use the `Pack` class, which is a subclass of the `System` class to pack a box of PPS molecules given a density. The system class creates the box and fill it with molecules, applies the force-field (if provided) to the system and creates
the initial state of the system in form a `gsd` snapshot. If force-field is provided, this class also gets the list of forces that defines the bonded and non-bonded interactions between the particles.

In this example, we pass the molecule object created in step 1 to pack a box with density=0.8. For the force-field, we use the pre-defined `OPLS` force-field class which includes all the parameters found in the OPLS xml force-field file.

In [ ]:
system = Pack(molecules=molecule, force_field=OPLS_AA_PPS(), density=0.8, r_cut=2.5, auto_scale=True)

We can obtain the `gsd` snapshot of the system by calling the `system.snapshot` attribute.


In [ ]:
system.hoomd_snapshot

We can also obtain the list of forces applied to the system by calling the `system.forces` attribute.

In [ ]:
system.hoomd_forcefield

### Step 3: Running the Simulation

Using the snapshot and forces provided by the system class, we can initialize the simulation. The `Simulation` class  logs snapshots of the simulation in form of a `gsd` trajectory file while running simulation. The `gsd_write_freq` specifies the frequency of saving snapshots into the gsd file. This class also logs other simulation data such as timestep, potential energy, kinetic temperature, pressure and volume into a text file. The frequency for logging these information can be set by `log_write_freq` parameter.

In [ ]:
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=system.hoomd_forcefield, gsd_write_freq=100, log_write_freq=100)

We can now run the simulation for 1000 time steps using the NVT ensemble at a given scaled temperature of 1.0.

In [ ]:
sim.run_NVT(n_steps=1000, kT=1.0, tau_kt=0.01)

The simulation class also allows user to run the simulation under different conditions such as NPT ensemble, NVE ensemble, Langevin dynamics. Checkout `hoomd_organics/base/simulation.py` for more functionalities.

In the rest of this tutorial, we will go through some of the features that are available in the `hoomd_organics` package that can be tailored to specific needs.

## Defining your own Molecule

You can define your own molecule in a couple of different ways:
- Using the SMILES string of the molecule
- Using the molecule file (accepted formats are: `.mol` and `.sdf`)
- Using a [`mbuild`](https://mbuild.mosdef.org/en/stable/) compound or a [`gmso`](https://gmso.mosdef.org/en/stable/) topology
- Define a subclass of the `Molecule` class

### Option 1: Using the SMILES string of the molecule

In [ ]:
# example of loading a molecule using the SMILES string
from hoomd_organics import Molecule

benzoic_acid_mol = Molecule(num_mols=20, smiles="c1cc(C(O)=O)ccc1")

We will use `mbuild` visualization function to visualize one of the 20 benzoic acid molecules.

In [ ]:
benzoic_acid_mol.molecules[0].visualize()

### Option 2: Using the molecule file

In [ ]:
# example of loading a molecule using the molecule file

phenol_mol = Molecule(num_mols=20, file="../hoomd_organics/assets/molecule_files/IPH.mol2")

In [ ]:
phenol_mol.molecules[0].visualize()

### Option 3: Using a [`mbuild`](https://mbuild.mosdef.org/en/stable/) compound or a [`gmso`](https://gmso.mosdef.org/en/stable/) topology

In [ ]:
# example of loading a molecule from mbuild compound or gmso topology
import mbuild as mb

mb_compound = mb.load("C1CCCCC1", smiles=True)

gmso_top = mb_compound.to_gmso()

benzene_mol = Molecule(num_mols=20, compound=mb_compound)
benzene_mol = Molecule(num_mols=20, compound=gmso_top)


### Option 4: Define a subclass of the `Molecule` class

Checkout some examples of polymer classes defined in `hoomd_organics/library/polymers.py`.

## Defining your own Forcefield
`hoomd-polymers` package has a list of pre-defined force-fields that can be used to initialize the system. If you have the `xml` file of the forcefield, you can use the `FF_from_file` class from `hoomd_organics.library` to create a force-field object.
You can also define your own forcefield by creating a subclass of the `foyer.Forcefield` class.


In [ ]:
# example of defining a force-field using the xml file
from hoomd_organics.library import FF_from_file

benzene_ff = FF_from_file(xml_file="../hoomd_organics/assets/forcefields/benzene_opls.xml")

Checkout `hoomd_organics/library/forcefields.py` for more some examples of defining a forcefield using a subclass of `foyer.Forcefield` for specific molecules.

## Defining your own System


`hoomd_organics` package has two methods of filling the box built in the `System` class: `Pack` and `Lattice`. (more info about pack and lattice?). Note that the base `System` class is considered an abstract class and cannot be called directly.

In [ ]:
# example of defining a system using the Lattice method

from hoomd_organics import Lattice
from hoomd_organics.library import OPLS_AA

benzene_mol = Molecule(num_mols=32, smiles="C1CCCCC1")

lattice = Lattice(
            molecules=[benzene_mol],
            force_field=OPLS_AA(),
            density=1.0,
            r_cut=2.5,
            x=1,
            y=1,
            n=4,
            auto_scale=True
        )

In [ ]:
lattice.system.visualize()

You can also define your own method of filling the box by creating a subclass of the `System` class. For example, one method of filling a box with two types of molecule is creating alternate layers of each molecule type.

##  Example of a system with multiple molecule types

The system class can take a list of different molecule types along with different forcefields. If all molecule types use the same forcefield, then you only need to pass the forcefield once.

In [ ]:
from hoomd_organics.library import OPLS_AA_DIMETHYLETHER
dimethylether_mol = Molecule(num_mols=20, smiles="COC")
pps_mol = PPS(num_mols=10, lengths=4)
multi_type_system = Pack(
    molecules=[dimethylether_mol, pps_mol],
    density=0.8,
    r_cut=2.5,
    force_field=[OPLS_AA_DIMETHYLETHER(), OPLS_AA_PPS()],
    auto_scale=True,
)

In [ ]:
multi_type_system.system.visualize()